In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import sys
import sqlite3

try:
    !{sys.executable} -m pip install html5lib
except Exception as e:
    print(e)
    print(e.__doc__)
    
import html5lib

     |████████████████████████████████| 112 kB 6.3 MB/s eta 0:00:01


In [2]:
def addCourseInfo(df,container_list):
    for s in container_list:
        section_list = s.findChildren('span',{'class':'section-id'})
        total_seats_list = s.findChildren('span',{'class':'total-seats-count'})
        open_seats_list = s.findChildren('span',{'class':'open-seats-count'})
        class_day_list = s.findChildren('div',{'class':'class-days-container'})
        for d in class_day_list:
            date_list = d.findChildren('span',{'class':'section-days'})
            start_time_list = d.findChildren('span',{'class':'class-start-time'})
            end_time_list = d.findChildren('span',{'class':'class-end-time'})
            building_list = d.findChildren('span',{'class':'building-code'})
            classroom_list = d.findChildren('span',{'class':'class-room'})
            if building_list and classroom_list and start_time_list and date_list:
                section_number = str(section_list[0].text).split()[0]
                students_number = int(total_seats_list[0].text)-int(open_seats_list[0].text)
                for i in range(len(building_list)):
                    if(str(building_list[i].text)!='TBA' and str(date_list[i].text)!='TBA'):
                        df2 = {'Major':major,'Course_id':course_id,'Section':section_number,'Building':building_list[i].text,'Classroom':classroom_list[i].text,'Section_days':date_list[i].text,'Start_time':start_time_list[i].text,'End_time':end_time_list[i].text,'Students_number':students_number}
                        df = df.append(df2, ignore_index = True)
    return df

In [3]:
URL = "https://app.testudo.umd.edu/soc/202101"
r = requests.get(url = URL)
soup = BeautifulSoup(r.content, 'html.parser')
t_list = soup.find_all('a',{'class':'clearfix'})
df = pd.DataFrame(columns=['Major','Course_id','Section','Building','Classroom','Section_days','Start_time','End_time','Students_number'])
for item in t_list:
    major = item['href'][-4:]
    major_url = 'https://app.testudo.umd.edu/soc/search?courseId='+major+'&sectionId=&termId=202101&_openSectionsOnly=on&creditCompare=%3E%3D&credits=0.0&courseLevelFilter=ALL&instructor=&facetoface=true&_facetoface=on&blended=true&_blended=on&_online=on&courseStartCompare=&courseStartHour=&courseStartMin=&courseStartAM=&courseEndHour=&courseEndMin=&courseEndAM=&teachingCenter=ALL&_classDay1=on&_classDay2=on&_classDay3=on&_classDay4=on&_classDay5=on'
    r = requests.get(url = major_url)
    major_soup = BeautifulSoup(r.content, 'html.parser')
    class_list = major_soup.find_all('div',{'class':'course'})
    for c in class_list:
        course_id = c['id']
        blended_section_container_list= c.find_all('div',{'class':'section delivery-blended'})
        f2f_section_container_list = c.find_all('div',{'class':'section delivery-f2f'})
        df = addCourseInfo(df,blended_section_container_list) 
        df = addCourseInfo(df,f2f_section_container_list) 
df

,Major,Course_id,Section,Building,Classroom,Section_days,Start_time,End_time,Students_number
0,AASP,AASP297,0101,TLF,1102,TuTh,2:00pm,3:15pm,0
1,AAST,AAST498Q,0101,CSI,1121,Tu,5:00pm,7:30pm,14
2,ANSC,ANSC103,0101,ANS,0108,M,10:00am,11:50am,19
3,ANSC,ANSC103,0102,ANS,1144,Tu,2:00pm,3:50pm,17
4,ANSC,ANSC245,0101,ANS,ONLINE,MW,3:00pm,3:50pm,10
...,...,...,...,...,...,...,...,...,...
658,TLPL,TLPL766,0101,ASY,2309,W,1:00pm,3:45pm,13
659,TLPL,TLPL788D,0101,TWS,0310,M,12:00pm,12:50pm,16
660,TLPL,TLPL788X,0101,ASY,2309,Tu,4:15pm,7:00pm,8
661,UMEI,UMEI004,0101,ESJ,2212,TuTh,9:00am,11:50am,1


In [4]:
con = sqlite3.connect("in_person_courses.sqlite")
cursor1 = con.cursor()
df.to_sql('in_person_courses_table', con, if_exists='replace', index=False)

# Showing all table in the new created databse
x = cursor1.execute("SELECT * FROM sqlite_master where type='table'")
for y in x.fetchall():
    print(y)

('table', 'in_person_courses_table', 'in_person_courses_table', 2, 'CREATE TABLE "in_person_courses_table" (\n"Major" TEXT,\n  "Course_id" TEXT,\n  "Section" TEXT,\n  "Building" TEXT,\n  "Classroom" TEXT,\n  "Section_days" TEXT,\n  "Start_time" TEXT,\n  "End_time" TEXT,\n  "Students_number" INTEGER\n)')


In [5]:
df.to_csv(r'in_person_courses.csv',index=False)